# Week 5 SQL: End-to-End Data Analysis

## Session Overview

In this  session, we will work through a data analysis workflow combining SQL and Pandas. Our goal is to analyze experimental data from a drug discovery program and identify the most promising compound-target combinations for further development.

We will simulate a scenario where you are a data scientist at a pharmaceutical company. Suppose you have access to a large database of experimental results, and your task is to prepare a report identifying promising drug candidates. The workflow will include data filtering with SQL, quality control checks, exploratory data analysis with Pandas, and visualization of results.

By the end of this session, you will have completed a full analysis pipeline that demonstrates when to use SQL versus Pandas and how to combine them effectively.

## The Business Problem

Your research team has been screening a library of chemical compounds against various protein targets associated with cancer. After months of experiments, you have accumulated:

- 500 chemical compounds with known properties
- 100 protein targets from different families
- Over 2000 experimental assay results measuring compound-protein binding

The project manager has asked you to identify promising candidates for the next phase of drug development. Specifically, you need to:

1. Filter for drug-like compounds that meet Lipinski's Rule of Five criteria
2. Identify compounds with strong and consistent binding (low IC50 values with low variability)
3. Focus on kinase targets, which are particularly relevant for cancer therapy
4. Perform quality control to ensure reliable results
5. Rank candidates and provide visualizations for the team meeting

Let's begin by setting up our environment and loading the data.

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create DuckDB connection
con = duckdb.connect(':memory:')

print("Environment ready. DuckDB version:", duckdb.__version__)

## Step 1: Load and Inspect the Data

In a real scenario, this data would come from a database or data warehouse. For our exercise, we will generate synthetic data. In practice, you would connect to your database using connection strings, but DuckDB makes it easy to work with both databases and in-memory data.

In [ ]:
np.random.seed(42)

# Compounds library (synthetic)
n_compounds = 500
compounds_data = {
    'compound_id': [f'CMPD_{i:04d}' for i in range(1, n_compounds + 1)],
    'compound_name': [f'MolSci-{i}' for i in range(1, n_compounds + 1)],
    'molecular_weight': np.random.uniform(150, 800, n_compounds).round(2),
    'log_p': np.random.uniform(-3, 7, n_compounds).round(2),
    'h_bond_donors': np.random.randint(0, 8, n_compounds),
    'h_bond_acceptors': np.random.randint(0, 12, n_compounds),
    'synthesis_date': [datetime(2023, 1, 1) + timedelta(days=int(x)) 
                       for x in np.random.uniform(0, 700, n_compounds)],
    'batch_quality': np.random.choice(['Excellent', 'Good', 'Fair'], n_compounds, p=[0.6, 0.3, 0.1])
}
compounds_df = pd.DataFrame(compounds_data)

# Protein targets (synthetic)
n_proteins = 100
protein_families = ['Kinase', 'GPCR', 'Ion Channel', 'Protease', 'Nuclear Receptor', 'Phosphatase']
proteins_data = {
    'protein_id': [f'PROT_{i:03d}' for i in range(1, n_proteins + 1)],
    'protein_name': [f'Target-{i}' for i in range(1, n_proteins + 1)],
    'protein_family': np.random.choice(protein_families, n_proteins),
    'disease_relevance': np.random.choice(['Cancer', 'Inflammation', 'Metabolic', 'Neurological'], 
                                          n_proteins, p=[0.4, 0.3, 0.2, 0.1]),
    'validation_status': np.random.choice(['Validated', 'Exploratory'], n_proteins, p=[0.7, 0.3])
}
proteins_df = pd.DataFrame(proteins_data)

# Assay results with sampled patterns
n_results = 2000
assay_results_data = {
    'assay_id': [f'ASSAY_{i:05d}' for i in range(1, n_results + 1)],
    'compound_id': np.random.choice(compounds_df['compound_id'].values, n_results),
    'protein_id': np.random.choice(proteins_df['protein_id'].values, n_results),
    'ic50_nm': np.random.lognormal(mean=4, sigma=2.5, size=n_results).round(2),
    'efficacy_percent': np.random.uniform(20, 100, n_results).round(1),
    'assay_date': [datetime(2024, 1, 1) + timedelta(days=int(x)) 
                   for x in np.random.uniform(0, 330, n_results)],
    'lab_technician': np.random.choice(['Alice Chen', 'Bob Kumar', 'Carol Martinez', 'David Wong'], n_results),
    'replicate_number': np.random.randint(1, 4, n_results)
}
assay_results_df = pd.DataFrame(assay_results_data)

# We add some intentional anomalies to demonstrate quality control as a use case
anomaly_indices = np.random.choice(n_results, size=20, replace=False)
assay_results_df.loc[anomaly_indices, 'ic50_nm'] *= 10  # Some spurious high values

print("Data loaded successfully!")
print(f"\nCompounds: {len(compounds_df):,} records")
print(f"Proteins: {len(proteins_df):,} records")
print(f"Assay Results: {len(assay_results_df):,} records")

## Step 2: Initial Data Exploration with SQL

Before diving into complex analysis, we should understand the basic characteristics of our data. Let's use SQL to quickly get summary statistics. This is often faster than Pandas for large datasets, and the queries are self-documenting.

In [ ]:
# Summary statistics for compounds
query = """
    SELECT 
        COUNT(*) as total_compounds,
        AVG(molecular_weight) as avg_mw,
        MIN(molecular_weight) as min_mw,
        MAX(molecular_weight) as max_mw,
        AVG(log_p) as avg_logp
    FROM compounds_df
"""

print("Compound Library Summary:")
print(con.execute(query).df())

# Distribution of compounds by batch quality
query = """
    SELECT 
        batch_quality,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 1) as percentage
    FROM compounds_df
    GROUP BY batch_quality
    ORDER BY count DESC
"""

print("\n\nBatch Quality Distribution:")
print(con.execute(query).df())

Now let's look at the protein target distribution and the overall assay statistics.

In [ ]:
# Protein target distribution
query = """
    SELECT 
        protein_family,
        disease_relevance,
        COUNT(*) as num_proteins
    FROM proteins_df
    GROUP BY protein_family, disease_relevance
    ORDER BY protein_family, num_proteins DESC
"""

print("Protein Target Distribution:")
print(con.execute(query).df())

# Overall assay statistics
query = """
    SELECT 
        COUNT(DISTINCT compound_id) as compounds_tested,
        COUNT(DISTINCT protein_id) as proteins_tested,
        COUNT(*) as total_assays,
        ROUND(AVG(ic50_nm), 2) as mean_ic50,
        ROUND(MEDIAN(ic50_nm), 2) as median_ic50,
        ROUND(STDDEV(ic50_nm), 2) as std_ic50
    FROM assay_results_df
"""

print("\n\nAssay Results Summary:")
print(con.execute(query).df())

Notice that the mean IC50 is much higher than the median. This suggests a right-skewed distribution, which is typical for biological activity data. A few inactive compounds with very high IC50 values pull the mean upward. This is why we often use the median or work with log-transformed IC50 values.

## Step 3: Filter for Drug-Like Compounds Using SQL

Lipinski's Rule of Five provides guidelines for drug-likeness based on molecular properties. Compounds should have molecular weight under 500 Da, log P under 5, fewer than 5 hydrogen bond donors, and fewer than 10 hydrogen bond acceptors (exceptions to more than one of these criteria breaks the Ro5). Let's use SQL to filter our compound library.

In [ ]:
# Apply Lipinski's Rule of Five
query = """
    SELECT 
        compound_id,
        compound_name,
        molecular_weight,
        log_p,
        h_bond_donors,
        h_bond_acceptors,
        batch_quality,
        CASE 
            WHEN molecular_weight <= 500 
                AND log_p <= 5 
                AND h_bond_donors <= 5 
                AND h_bond_acceptors <= 10 
            THEN 'Pass'
            ELSE 'Fail'
        END as lipinski_status
    FROM compounds_df
    WHERE batch_quality IN ('Excellent', 'Good')
"""

druglike_compounds = con.execute(query).df()

print(f"Total compounds after quality filter: {len(druglike_compounds):,}")
print(f"Drug-like compounds (pass Lipinski): {(druglike_compounds['lipinski_status'] == 'Pass').sum():,}")
print(f"Percentage passing: {100 * (druglike_compounds['lipinski_status'] == 'Pass').sum() / len(druglike_compounds):.1f}%")

print("\nSample of drug-like compounds:")
print(druglike_compounds[druglike_compounds['lipinski_status'] == 'Pass'].head(10))

By filtering at the SQL level, we reduce the amount of data we need to work with in subsequent steps. This is a key principle: use SQL to filter large datasets early in your pipeline, then bring the smaller result set into Pandas for detailed analysis.

## Step 4: Quality Control - Identifying Reliable Measurements

Before analyzing binding data, we need to ensure data quality. Let's look for potential issues such as compounds or proteins with high measurement variability and outlier values that might indicate experimental errors.

In [ ]:
# Check for compounds with high variability in measurements
query = """
    WITH compound_variability AS (
        SELECT 
            compound_id,
            COUNT(*) as num_measurements,
            AVG(ic50_nm) as mean_ic50,
            STDDEV(ic50_nm) as std_ic50,
            MIN(ic50_nm) as min_ic50,
            MAX(ic50_nm) as max_ic50
        FROM assay_results_df
        GROUP BY compound_id
        HAVING COUNT(*) >= 3
    )
    SELECT 
        compound_id,
        num_measurements,
        ROUND(mean_ic50, 2) as mean_ic50,
        ROUND(std_ic50, 2) as std_ic50,
        ROUND(std_ic50 / mean_ic50, 2) as coefficient_of_variation,
        ROUND(min_ic50, 2) as min_ic50,
        ROUND(max_ic50, 2) as max_ic50
    FROM compound_variability
    WHERE std_ic50 / mean_ic50 > 1.0  -- High coefficient of variation
    ORDER BY std_ic50 / mean_ic50 DESC
    LIMIT 15
"""

high_variability = con.execute(query).df()
print("Compounds with high measurement variability (CV > 1.0):")
print(high_variability)

The coefficient of variation (CV) is the standard deviation divided by the mean. A CV greater than 1.0 indicates that the standard deviation is larger than the mean, suggesting inconsistent measurements. These compounds may have quality issues such as poor solubility, degradation, or assay interference.

For our analysis, we will focus on compounds with reliable, reproducible measurements.

In [ ]:
# Identify outlier measurements using statistical approach
query = """
    WITH compound_protein_stats AS (
        SELECT 
            compound_id,
            protein_id,
            AVG(ic50_nm) as mean_ic50,
            STDDEV(ic50_nm) as std_ic50,
            COUNT(*) as num_replicates
        FROM assay_results_df
        GROUP BY compound_id, protein_id
        HAVING COUNT(*) >= 2
    )
    SELECT 
        a.assay_id,
        a.compound_id,
        a.protein_id,
        a.ic50_nm,
        s.mean_ic50,
        s.std_ic50,
        ROUND((a.ic50_nm - s.mean_ic50) / s.std_ic50, 2) as z_score
    FROM assay_results_df a
    INNER JOIN compound_protein_stats s
        ON a.compound_id = s.compound_id 
        AND a.protein_id = s.protein_id
    WHERE ABS((a.ic50_nm - s.mean_ic50) / s.std_ic50) > 2.5
        AND s.std_ic50 > 0
    ORDER BY ABS((a.ic50_nm - s.mean_ic50) / s.std_ic50) DESC
"""

outliers = con.execute(query).df()
print(f"\n\nOutlier measurements (|z-score| > 2.5): {len(outliers)}")
print(outliers.head(15))

We identified measurements that are more than 2.5 standard deviations away from the mean for their compound-protein pair. In a real analysis, we would investigate these outliers: are they experimental errors, or do they represent genuine biological variability?

For this exercise, we will proceed by excluding extreme outliers from our analysis to focus on reliable data.

## Step 5: Focus on Cancer-Relevant Kinase Targets

Our project manager specifically asked for cancer-relevant kinase inhibitors. Let's use SQL to join our tables and filter for relevant data, then calculate aggregate statistics for each compound-protein pair.

In [ ]:
# Create a clean dataset of cancer-relevant kinase assays
query = """
    WITH reliable_measurements AS (
        SELECT 
            compound_id,
            protein_id,
            AVG(ic50_nm) as mean_ic50,
            STDDEV(ic50_nm) as std_ic50,
            COUNT(*) as num_replicates,
            AVG(efficacy_percent) as mean_efficacy
        FROM assay_results_df
        WHERE ic50_nm < 10000  -- Exclude extremely weak binders
        GROUP BY compound_id, protein_id
        HAVING COUNT(*) >= 2  -- Require at least 2 replicates
    )
    SELECT 
        c.compound_id,
        c.compound_name,
        c.molecular_weight,
        c.log_p,
        c.batch_quality,
        p.protein_id,
        p.protein_name,
        p.protein_family,
        p.disease_relevance,
        rm.mean_ic50,
        rm.std_ic50,
        rm.num_replicates,
        rm.mean_efficacy,
        CASE 
            WHEN c.molecular_weight <= 500 
                AND c.log_p <= 5 
                AND c.h_bond_donors <= 5 
                AND c.h_bond_acceptors <= 10 
            THEN 1
            ELSE 0
        END as passes_lipinski
    FROM reliable_measurements rm
    INNER JOIN compounds_df c ON rm.compound_id = c.compound_id
    INNER JOIN proteins_df p ON rm.protein_id = p.protein_id
    WHERE p.protein_family = 'Kinase'
        AND p.disease_relevance = 'Cancer'
        AND c.batch_quality IN ('Excellent', 'Good')
"""

kinase_data = con.execute(query).df()

print(f"Cancer-relevant kinase data after quality filters:")
print(f"  Total compound-protein pairs: {len(kinase_data):,}")
print(f"  Unique compounds: {kinase_data['compound_id'].nunique():,}")
print(f"  Unique proteins: {kinase_data['protein_id'].nunique():,}")
print(f"  Compounds passing Lipinski: {kinase_data['passes_lipinski'].sum():,}")

print("\nFirst few rows:")
print(kinase_data.head(10))

We have successfully used SQL to perform complex filtering and joining across three tables, calculating aggregate statistics and applying multiple criteria. The result is a clean, focused dataset ready for detailed analysis in Pandas.

Notice how SQL made this multi-step filtering process clear and efficient. The equivalent Pandas code would be more verbose and slower for large datasets.

## Step 6: Exploratory Data Analysis with Pandas

Now that we have a focused dataset, let's use Pandas for exploratory analysis. We will examine the distribution of IC50 values, look for correlations between molecular properties and activity, and identify the most promising compounds.

In [ ]:
print("IC50 Distribution Statistics:")
print(kinase_data['mean_ic50'].describe())

print("\n\nEfficacy Distribution Statistics:")
print(kinase_data['mean_efficacy'].describe())

kinase_data['potency_category'] = pd.cut(
    kinase_data['mean_ic50'],
    bins=[0, 10, 100, 1000, float('inf')],
    labels=['Very Potent (<10 nM)', 'Potent (10-100 nM)', 
            'Moderate (100-1000 nM)', 'Weak (>1000 nM)']
)

print("\n\nDistribution by Potency Category:")
print(kinase_data['potency_category'].value_counts().sort_index())

The data shows that most compounds fall in the moderate to weak potency range, which is typical for early-stage screening. However, we do have some very potent compounds with IC50 values below 10 nM, which are excellent starting points for drug development.

In [ ]:
# Analyze relationship between molecular properties and potency
# Focus on drug-like compounds
druglike_kinase = kinase_data[kinase_data['passes_lipinski'] == 1].copy()

# Add log-transformed IC50 for better visualization
druglike_kinase['log_ic50'] = np.log10(druglike_kinase['mean_ic50'])

print(f"Drug-like kinase inhibitors: {len(druglike_kinase):,} compound-protein pairs")
print(f"Unique drug-like compounds: {druglike_kinase['compound_id'].nunique():,}")

correlation_data = druglike_kinase[['molecular_weight', 'log_p', 'log_ic50', 'mean_efficacy']].corr()
print("\n\nCorrelation Matrix:")
print(correlation_data.round(3))

The correlation analysis helps us understand relationships between molecular properties and biological activity. In drug discovery, we often look for compounds that balance potency (low IC50) with drug-like properties.

A weak negative correlation between log P and log IC50 might suggest that more lipophilic compounds show better binding, but we need to be careful not to over-interpret small correlations.

In [ ]:
# Identify top candidates: most potent compounds with good properties
top_candidates = (druglike_kinase
    .groupby('compound_id')
    .agg({
        'compound_name': 'first',
        'molecular_weight': 'first',
        'log_p': 'first',
        'mean_ic50': 'mean',  # Average across all kinases tested
        'mean_efficacy': 'mean',
        'protein_id': 'count',  # Number of kinases tested
        'std_ic50': 'mean'  # Average variability
    })
    .rename(columns={'protein_id': 'num_kinases_tested'})
    .sort_values('mean_ic50')
    .head(20)
)

top_candidates['avg_cv'] = top_candidates['std_ic50'] / top_candidates['mean_ic50']

print("Top 20 Most Potent Drug-Like Compounds:")
print(top_candidates)

These top candidates combine strong potency with drug-like properties and have been tested against multiple kinase targets. Compounds tested against more targets give us confidence in their reproducibility and selectivity profile.

## Step 7: Selectivity Analysis

In drug discovery, selectivity is crucial. A compound that inhibits many kinases may cause side effects, while a highly selective compound may be safer. Let's analyze the selectivity profile of our top candidates.

In [ ]:
# For each top compound, see how many kinases it potently inhibits
selectivity_threshold = 100  # Consider IC50 < 100 nM as "potent inhibition"

selectivity_analysis = []

for compound_id in top_candidates.index[:10]:  # Analyze top 10 compounds
    compound_data = kinase_data[kinase_data['compound_id'] == compound_id]
    
    total_tested = len(compound_data)
    potent_hits = (compound_data['mean_ic50'] < selectivity_threshold).sum()
    best_ic50 = compound_data['mean_ic50'].min()
    worst_ic50 = compound_data['mean_ic50'].max()
    
    selectivity_analysis.append({
        'compound_id': compound_id,
        'compound_name': compound_data['compound_name'].iloc[0],
        'kinases_tested': total_tested,
        'potent_hits': potent_hits,
        'selectivity_ratio': potent_hits / total_tested,
        'best_ic50': best_ic50,
        'worst_ic50': worst_ic50,
        'dynamic_range': worst_ic50 / best_ic50
    })

selectivity_df = pd.DataFrame(selectivity_analysis).sort_values('selectivity_ratio')

print("Selectivity Analysis (Top 10 Compounds):")
print(selectivity_df.to_string(index=False))

A compound with a low selectivity ratio (potently inhibits only a few of the kinases tested) is more selective, which is generally desirable to minimize off-target effects. The dynamic range (ratio of worst to best IC50) also indicates selectivity: a large range means the compound discriminates well between different kinases.

## Step 8: Visualization

Let's create visualizations to present our findings. Good visualizations are essential for communicating results to non-technical stakeholders.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Cancer-Relevant Kinase Inhibitor Analysis', fontsize=16, fontweight='bold')

# Plot 1: Distribution of IC50 values (log scale)
ax1 = axes[0, 0]
ax1.hist(np.log10(kinase_data['mean_ic50']), bins=40, edgecolor='black', alpha=0.7)
ax1.axvline(np.log10(100), color='red', linestyle='--', linewidth=2, label='100 nM threshold')
ax1.set_xlabel('log10(IC50) in nM', fontsize=11)
ax1.set_ylabel('Frequency', fontsize=11)
ax1.set_title('Distribution of IC50 Values', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Molecular weight vs IC50 for drug-like compounds
ax2 = axes[0, 1]
scatter = ax2.scatter(
    druglike_kinase['molecular_weight'], 
    druglike_kinase['log_ic50'],
    c=druglike_kinase['mean_efficacy'],
    cmap='viridis',
    alpha=0.6,
    s=30
)
ax2.set_xlabel('Molecular Weight (Da)', fontsize=11)
ax2.set_ylabel('log10(IC50) in nM', fontsize=11)
ax2.set_title('Molecular Weight vs Potency', fontsize=12, fontweight='bold')
cbar = plt.colorbar(scatter, ax=ax2)
cbar.set_label('Efficacy (%)', fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Potency categories
ax3 = axes[1, 0]
potency_counts = kinase_data['potency_category'].value_counts().sort_index()
colors_potency = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c']
ax3.bar(range(len(potency_counts)), potency_counts.values, color=colors_potency, edgecolor='black')
ax3.set_xticks(range(len(potency_counts)))
ax3.set_xticklabels(potency_counts.index, rotation=45, ha='right', fontsize=9)
ax3.set_ylabel('Number of Compound-Protein Pairs', fontsize=11)
ax3.set_title('Distribution by Potency Category', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Top 10 compounds by average IC50
ax4 = axes[1, 1]
top_10_plot = top_candidates.head(10).sort_values('mean_ic50', ascending=False)
bars = ax4.barh(range(len(top_10_plot)), top_10_plot['mean_ic50'], color='#3498db', edgecolor='black')
ax4.set_yticks(range(len(top_10_plot)))
ax4.set_yticklabels([name.split('-')[1] for name in top_10_plot['compound_name']], fontsize=9)
ax4.set_xlabel('Mean IC50 (nM)', fontsize=11)
ax4.set_title('Top 10 Most Potent Compounds', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='x')

for i, (idx, row) in enumerate(top_10_plot.iterrows()):
    ax4.text(row['mean_ic50'] + 1, i, f"{row['mean_ic50']:.1f}", 
             va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\nVisualizations created successfully!")

We have identified a set of potent, drug-like kinase inhibitors with IC50 values well below 100 nM. The molecular weight versus potency plot shows that potent compounds can be found across the drug-like molecular weight range, and higher efficacy compounds are distributed throughout the potency spectrum.

## Step 9: Generate Final Report Using SQL + Pandas

Let's create a final summary table that we can present. This will combine SQL for initial filtering and Pandas for final formatting.

In [ ]:
# Use SQL to get comprehensive statistics, then format in Pandas
query = """
    WITH compound_summary AS (
        SELECT 
            c.compound_id,
            c.compound_name,
            c.molecular_weight,
            c.log_p,
            AVG(a.mean_ic50) as avg_ic50_all_kinases,
            MIN(a.mean_ic50) as best_ic50,
            COUNT(DISTINCT a.protein_id) as num_kinases_tested,
            AVG(a.mean_efficacy) as avg_efficacy,
            SUM(CASE WHEN a.mean_ic50 < 100 THEN 1 ELSE 0 END) as potent_kinase_count
        FROM kinase_data a
        INNER JOIN compounds_df c ON a.compound_id = c.compound_id
        WHERE a.passes_lipinski = 1
        GROUP BY c.compound_id, c.compound_name, c.molecular_weight, c.log_p
        HAVING COUNT(DISTINCT a.protein_id) >= 3  -- Tested against at least 3 kinases
    )
    SELECT 
        compound_id,
        compound_name,
        ROUND(molecular_weight, 1) as mw,
        ROUND(log_p, 2) as logp,
        ROUND(avg_ic50_all_kinases, 2) as avg_ic50,
        ROUND(best_ic50, 2) as best_ic50,
        num_kinases_tested,
        ROUND(avg_efficacy, 1) as avg_efficacy,
        potent_kinase_count,
        ROUND(100.0 * potent_kinase_count / num_kinases_tested, 1) as selectivity_pct
    FROM compound_summary
    ORDER BY avg_ic50_all_kinases
    LIMIT 15
"""

final_report = con.execute(query).df()

final_report['priority_score'] = (
    (1000 / final_report['avg_ic50']) * 0.5 +  # Potency weight: 50%
    (final_report['avg_efficacy'] / 100) * 0.3 +  # Efficacy weight: 30%
    ((100 - final_report['selectivity_pct']) / 100) * 0.2  # Selectivity weight: 20%
)

final_report['priority_rank'] = final_report['priority_score'].rank(ascending=False, method='dense').astype(int)
final_report = final_report.sort_values('priority_rank')

print("=" * 100)
print("FINAL REPORT: Top 15 Drug Candidates for Cancer-Relevant Kinase Inhibition")
print("=" * 100)
print(final_report.to_string(index=False))

## Report Summary

The priority score combines three key factors with arbitray weights (let's assume this is just what our project manager has communicated to us):

- Potency (50% weight): Lower IC50 values indicate stronger binding
- Efficacy (30% weight): Higher efficacy suggests better therapeutic potential
- Selectivity (20% weight): Lower selectivity percentage means the compound is more selective (fewer off-target kinases)

The top-ranked compounds represent the best balance of these properties and should be prioritized for further development.

## Step 10: Additional Analysis - Temporal Trends

As a bonus analysis, let's examine whether compound synthesis and testing patterns have changed over time. This can reveal insights about research focus and experimental throughput.

In [ ]:
# Analyze testing activity over time using SQL
query = """
    WITH monthly_activity AS (
        SELECT 
            DATE_TRUNC('month', a.assay_date) as test_month,
            COUNT(*) as num_tests,
            COUNT(DISTINCT a.compound_id) as unique_compounds,
            AVG(a.mean_ic50) as avg_ic50_that_month
        FROM kinase_data a
        GROUP BY DATE_TRUNC('month', a.assay_date)
    )
    SELECT 
        test_month,
        num_tests,
        unique_compounds,
        ROUND(avg_ic50_that_month, 2) as avg_ic50
    FROM monthly_activity
    ORDER BY test_month
"""

temporal_data = con.execute(query).df()
temporal_data['test_month'] = pd.to_datetime(temporal_data['test_month'])

fig, axes = plt.subplots(2, 1, figsize=(12, 8))
fig.suptitle('Temporal Trends in Kinase Testing', fontsize=14, fontweight='bold')

# Plot testing volume over time
ax1 = axes[0]
ax1.plot(temporal_data['test_month'], temporal_data['num_tests'], 
         marker='o', linewidth=2, markersize=6, color='#3498db')
ax1.set_ylabel('Number of Tests', fontsize=11)
ax1.set_title('Testing Activity Over Time', fontsize=12)
ax1.grid(True, alpha=0.3)

# Plot average IC50 over time
ax2 = axes[1]
ax2.plot(temporal_data['test_month'], temporal_data['avg_ic50'], 
         marker='s', linewidth=2, markersize=6, color='#e74c3c')
ax2.set_ylabel('Average IC50 (nM)', fontsize=11)
ax2.set_xlabel('Month', fontsize=11)
ax2.set_title('Average Potency Over Time', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Temporal analysis complete!")

This temporal analysis can reveal important patterns. For example, if average IC50 decreases over time, it suggests that the research team is successfully optimizing compounds or focusing on more promising chemical series.

## Conclusion

Congratulations! In this hands-on session, we completed a full data analysis workflow combining SQL and Pandas:

**Step 1-2**: We used SQL for initial data exploration and summary statistics, taking advantage of SQL's ability to quickly aggregate large datasets.

**Step 3**: We applied Lipinski's Rule of Five using SQL's CASE statements to filter for drug-like compounds early in our pipeline.

**Step 4**: We performed quality control by identifying high-variability compounds and outlier measurements using SQL's statistical functions and subqueries.

**Step 5**: We combined multiple tables with INNER JOINs and applied complex filters to focus on cancer-relevant kinase inhibitors, demonstrating SQL's strength in relational data operations.

**Step 6-7**: We switched to Pandas for detailed exploratory analysis, creating new features and performing selectivity analysis that would be more cumbersome in pure SQL.

**Step 8-9**: We created visualizations and a final report, using SQL for the final data preparation and Pandas for ranking and formatting.

**Step 10**: We demonstrated how SQL and Pandas can work together for temporal analysis.

## When to Use SQL vs Pandas?

The best approach combined both tools:
- Use SQL when you need to filter large datasets before loading into memory, when joining multiple tables from a database, when performing aggregations that can be indexed efficiently in a database, or when writing self-documenting queries that colleagues can understand.
- Use Pandas when performing complex transformations specific to Python, when integrating with Python visualization and statistical libraries, when working with data that is already in memory, or when iterative exploratory analysis requires flexibility.

In this hybrid approach, we leveraged SQL's efficiency for data preparation, then used Pandas' flexibility for detailed analysis and visualization.

Remember that the goal is not to use one tool exclusively, but to understand the strengths of each and combine them effectively. SQL excels at filtering and aggregating large datasets, while Pandas provides unmatched flexibility for exploratory analysis and integration with Python's scientific computing ecosystem.

As you continue in your data science journey, you will develop intuition for when to reach for SQL versus Pandas bersus something else, and how to structure workflows that leverage multiple tools optimally.